# Wiki Summarizer

This Project takes the name of a topic as input, and checks if the corresponding wiki-page exists. If it does, it parses the web page, and outputs a summary created using the GPT-4o-mini model. 

Concepts used: 
- Web Scraping via Beautiful Soup
- User and System Prompts
- Use of Open AI GPT-4o-mini via API key

In [0]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [0]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [0]:
# creating an instance
openai = OpenAI()

In [0]:
def isWiki(url):
    """
    Check whether a Wikipedia page exists for a given topic, and 
    returns a Boolean value.
    """
    response = requests.get(url)

    if response.status_code != 200:
        return False
        
    return True

In [0]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [0]:
system_prompt = "You are an educational assistant tasked with helping users understand topics\
by providing succinct and clear summaries of requested data. Ignore navigation-related text\
and provide answers in markdown format"

In [0]:
# A function that writes a User Prompt that asks for summaries of wiki pages:

def user_prompt_for(wiki):
    user_prompt = f"You are looking at a Wikipedia page titled {wiki.title}"
    user_prompt += "\nThe contents of this page is as follows; \
please provide a short summary of this website in markdown.\n"
    user_prompt += wiki.text
    return user_prompt

In [0]:
def messages_for(wiki):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(wiki)}
    ]

In [0]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [0]:
def display_summary(topic):
    url = f"https://en.wikipedia.org/wiki/{topic}"
    if isWiki(url):
        summary = summarize(url)
        display(Markdown(summary))
    else:
        print('A Wikipedia page does not exist for this topic')
        

In [0]:
topic = input('Enter the name of Wikipedia page for which you would like a summary: ').strip()
display_summary(topic)